<a href="https://colab.research.google.com/github/fernandodeeke/can2025/blob/main/LU_Cholesky_QR_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1></h1></center>
<center><h1>Cálculo Numérico CAN0001</h1></center>
<center><h2>2025/2</h2></center>
<center><h3>Fernando Deeke Sasse</h3></center>
<center><h3>CCT - UDESC</h3></center>
<center><h2>Fatoração LU, Cholesky e QR</h2></center>

### 1.  Introdução

É possível mostrar q qualquer matriz não singular $\mathbf{A}$ pode ser decomposta uma matriz triangular inferior $\mathbf{L}$, e matriz triangular superior $\mathbf{U}$, ou seja,

$$ A = LU, $$

com

\begin{equation}
\mathbf{A} = \begin{pmatrix}a_{11}&a_{12}&\ldots&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
\mathbf{L} = \begin{pmatrix}l_{11}&0&\ldots&0\\ l_{21}&\ddots& &0\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},\\
\mathbf{U} = \begin{pmatrix}u_{11}&u_{12}&\ldots&u_{1n}\\0&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\0&0&...&u_{nn}\end{pmatrix}.
\end{equation}

O procedimento, como veremos a seguir, consiste no uso da eliminação gaussiana.
Este é um dos métodos mais utilizados na prática para sistema lineares não esparsos (sistemas esparsos, ou seja com uma matriz de coefientes com a maior parte de componentes nulas, serão tratados mais adiante por meio de métodos iterativos.

A ideia consiste no seguinte. Uma vez obtida a fatoração na forma $A = LU$ o sistema $AX=B$ passa a ser ter a forma $LUX=B$. Definindo $Y=UX$ o sistema agora tem a forma $LY=B$. Este sistema pode ser resolvido para $Y$ usando substituição avançada. Ou seja, determinamos $y_1$ a partir da primeira linha, $y_2$ a partir da segunda, e assim por diante. Uma vez obtido $Y$ o próximo passo consiste em resolver o sistema $UX=Y$, que pode ser resolvido para $X$ usando retrosubstituição.

A vantagem deste procedimento torna-se aparente quando temos que resolver diversos sistemas de equações que têm a mesma matriz de coeficientes,

$$
AX_1=B_1, \qquad AX_2=B_2,\quad \cdots \quad,\, AX_N=B_N\,.
$$

Outra aplicação consiste na inversão de matrizes. Para inverter uma matriz $A$, de ordem $n \times n$, devemos encontrar uma matriz inversa de $A$, denominada $A^{-1}$, que satisfaz a

$$
A A^{-1} = I,
$$
sendo $I$ a matriz identidade, $n \times n$.

Reescrevamos $I$ e $A^{-1}$ em termos de matrizes coluna, ou seja,  $I = [e_1 | e_2 | \cdots e_n]$, e  $ A^{-1} = [v_1 | v_2 | \cdots v_n]$. A determinação da matriz inversa agora consiste na resolução de $n$ sistemas lineares da forma

$$
A v_1 = e_1\,, \quad Av_2 = e_2\,,\cdots \,,\, Av_n=e_n\,.
$$

A eliminação Gaussiana tem um custo operacional da ordem de $O(n^3)$ operações. Este é o mesmo custo da operação de fatoração de matrizes, com a diferença de que na fatoração LU a operação é feita somente uma vez no caso de termos que resolver vários sistemas com a mesma matriz de coeficientes. O custo das operações de substituição atrasada (retrosubstituição) e avançada é $O(n^2)$.

A fatoração LU não é única, mas há três tipos que são mais utilizados:

1. Fatoração de Doolittle: $l_1 = l_2 = \ldots = l_n = 1$.
2. Fatoração de Crout: $u_1 = u_2 = \ldots = u_n = 1$.
3. Fatoração de Cholesky: $L = U^T$ (quando a matriz $A$ é simétrica).

Veremos a seguir o procedimento computacional para a fatoração de Doolittle.

### 2. Procedimento operacional para fatoração de Doolittle

Simplesmente realizamos a eliminação gaussiana sobre a matriz A para obter uma matriz triangular superior. Os multiplicadores $m_{ij}$ utilizados na eliminação dos elementos $a_{ij}$ de $A$ são exatamente as componentes abaixo da diagonal da matriz $L$, ou seja, $l_{ij}=m_{ij}$.

Vejamos um exemplo, executando o procedimento passo a passo.

Usaremos uma função que realiza operações elementares de linha:

In [69]:
def soma_linha(A,k,i,j):
    "Soma k vezes a linha j à linha i na matriz A"
    n=A.shape[0]
    E=np.eye(n)
    if i == j:
        E[i,i] = k+1
    else:
        E[i,j] = k
    return E@A

In [70]:
import numpy as np

Consideremos o sistema $AX=B$ com

In [71]:
A = np.array([[2.,4.,5.], [5.,9.,-3.],[3.,5.,1.]])
A

array([[ 2.,  4.,  5.],
       [ 5.,  9., -3.],
       [ 3.,  5.,  1.]])

In [72]:
B = np.array([[2,3,1]])
B.T

array([[2],
       [3],
       [1]])

Realizemos o procedimento de eliminação gaussiana armazenando os multiplicadores.

In [73]:
m10 = 5./2.
A1 = soma_linha(A,-m10,1,0)
A1

array([[  2. ,   4. ,   5. ],
       [  0. ,  -1. , -15.5],
       [  3. ,   5. ,   1. ]])

In [74]:
m20 = 3./2.
A2 = soma_linha(A1,-m20,2,0)
A2

array([[  2. ,   4. ,   5. ],
       [  0. ,  -1. , -15.5],
       [  0. ,  -1. ,  -6.5]])

In [75]:
m21 = 1.
A3 = soma_linha(A2,-m21,2,1)
A3

array([[  2. ,   4. ,   5. ],
       [  0. ,  -1. , -15.5],
       [  0. ,   0. ,   9. ]])

A matriz escalonada é a matriz U

In [76]:
U = A3
U

array([[  2. ,   4. ,   5. ],
       [  0. ,  -1. , -15.5],
       [  0. ,   0. ,   9. ]])

A matriz L é aquela com os multiplicadores abaixo da diagonal:

In [77]:
L=np.diag(np.array([1.,1.,1.]))
L[1,0]=m10
L[2,0]=m20
L[2,1]=m21
L

array([[1. , 0. , 0. ],
       [2.5, 1. , 0. ],
       [1.5, 1. , 1. ]])

Podemos verificar que realmente $LU=A$:

In [78]:
L@U-A

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

Definimos a função para substituição avançada:

In [79]:
def GaussAv(a, b):
    n = len(b)
    x = np.zeros(n)
    x[0] = b[0].item()  # ou float(b[0])
    for k in range(1, n):
        x[k] = (b[k] - np.dot(a[k, 0:k], x[0:k])).item()
    return np.transpose([x])

Resolvemos $LY=B$:

In [80]:
Y = GaussAv(L,B.T)
Y

array([[ 2.],
       [-2.],
       [ 0.]])

Para resolver $UX=Y$ devemos usar a retrosubstituição:

In [81]:
def GaussRetro(a, b):
    n = len(b)
    x = np.zeros(n)
    for k in range(n - 1, -1, -1):
        x[k] = ((b[k] - np.dot(a[k, k + 1:], x[k + 1:])) / a[k, k]).item()
    return np.transpose([x])  # vetor coluna

In [82]:
X=GaussRetro(U,Y)
X

array([[-3.],
       [ 2.],
       [ 0.]])

Podemos verificar este resultado:

In [83]:
A@X-B.T

array([[0.],
       [0.],
       [0.]])

### 3.  Pseudocódigo para a fatoração LU de Doolittle

Consideremos as matrizes:

\begin{equation}
\mathbf{A} = \begin{pmatrix}a_{11}&a_{12}&\ldots&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&...&...&a_{nn}\end{pmatrix},
\mathbf{L} = \begin{pmatrix}l_{11}&l_{12}&\ldots&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&...&...&l_{nn}\end{pmatrix},\\
\mathbf{U} = \begin{pmatrix}u_{11}&u_{12}&\ldots&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&...&...&u_{nn}\end{pmatrix}
\end{equation}

| Passos | |
|--: | :-- |
| 1. | Inicialize $\mathbf{L}$ como uma matriz identidade, $\mathbf{I}$ de dimensão $n\times n$ e $\mathbf{U = A}$.
| 2. | Para $i = 1, \ldots, n$ realize passo 3
| 3. | $\phantom{--}$ For $j=i+1, \ldots, n$ realize passos 4-5
| 4. | $\phantom{----}$ Faça $l_{ji}=u_{ji}/u_{ii}$
| 5. | $\phantom{----}$ Determine $U_j = (U_j-l_{ji}U_i)$ (sendo $U_i, U_j$ as linhas $i$ e $j$ da matriz $\mathbf{U}$, respectivamente.)

### 4. Função em Python para fatoração de Doolittle

Um modo de definir tal função é o seguinte:

In [84]:
def lu1(A):
    A = A.copy().astype(float)
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros_like(A)

    for k in range(n):
        if np.isclose(A[k, k], 0):
            raise ValueError("Zero na diagonal. A fatoração sem pivotação falhará.")

        U[k, k:] = A[k, k:] - L[k, :k] @ U[:k, k:]
        for i in range(k + 1, n):
            L[i, k] = (A[i, k] - L[i, :k] @ U[:k, k]) / U[k, k]

    return L, U

Vejamos um exemplo:

In [85]:
A = np.array([[2, 3, 1],
              [4, 7, 5],
              [6, 18, 22]], dtype=float)

In [86]:
L, U =lu1(A)
print(L)
print(U)

[[1. 0. 0.]
 [2. 1. 0.]
 [3. 9. 1.]]
[[ 2.  3.  1.]
 [ 0.  1.  3.]
 [ 0.  0. -8.]]


Verifiquemos o resultado:

In [87]:
np.linalg.norm(L@U-A, ord=np.inf)

np.float64(0.0)

Podemos agora escrever a função completa que resolve o sistema $AX=b$:

In [88]:
def lu_solve1(A, b):
    L, U = lu1(A)
    y = GaussAv(L, b)
    return GaussRetro(U, y)

Testemos a função no mesmo sistema usado anteriormente:

In [89]:
A = np.array([[2.,4.,5.], [5.,9.,-3.],[3.,5.,1.]])
B = np.array([2,3,1])
B = b.reshape(n,1)

In [90]:
X = lu_solve1(A,B)
X

IndexError: index 3 is out of bounds for axis 0 with size 3

In [91]:
A@X-B

ValueError: operands could not be broadcast together with shapes (3,1) (100,1) 

Este é o resultado correto. Vejamos um sistema aleatório:

In [ ]:
# Tamanho do sistema
n = 9

# Gerar uma matriz A aleatória (com valores flutuantes)
A = np.random.rand(n, n)

# Gerar um vetor b aleatório
b = np.random.rand(n)
B = b.reshape(n,1)

In [ ]:
X = lu_solve1(A,B)
X

array([[-0.30953833],
       [-1.18825915],
       [-1.8930222 ],
       [-2.18751174],
       [ 3.03181636],
       [ 0.61582566],
       [ 1.85182772],
       [-2.09387675],
       [ 0.97232374]])

### 4. Fatoração PLU

O algoritmo de fatoração LU que vimos acima falhará se em algum ponto tivermos  $a_{ii}=0$.  Para evitar tal problema, devemos modificar o algoritmos para que, quando tal evento ocorrer haja uma troca desta linha com outra com valor não nulo na coluna $i$.

Várias estratégias podem ser utilizadas, tais como pivotação parcial de linhas ou colunas. No entanto, por simplicidade faremos aqui somente uma troca de qualquer linha que contenha um elemento nulo na diagonal pela primeira linha abaixo desta que contenha um elemento não nulo nesta coluna.

Como estamos trocando linhas somente da matriz de coeficientes, é necessário manter um histórico das trocas de linhas para depois realizá-las no lado direito de uma equação $AX=B$. Para isso criamos uma matriz de permutação multiplicativa $P$ que consiste de uma matriz identidade com as mesmas linhas trocadas.  No final, teremos uma solução para o sistema $PA X = PB$.


### 5. Pseudocódigo para Fatoração PLU

Construiremos agora o algoritmo de fatoração LU que realiza pivotação. O método é chamado PLU, pois uma vez feita a fatoração a matriz de permutação de linhas $P$ deve ser tal que  $PA = LU$. O código abaixo é semelhante àquele para a fatoração LU.

Consideremos as matrizes:

\begin{equation}
\mathbf{A} = \begin{bmatrix}a_{11}&a_{12}&\ldots&a_{1n}\\a_{21}&\ddots& &a_{2n}\\\vdots& &\ddots& \vdots\\a_{n1}&\ldots&\ldots&a_{nn}\end{bmatrix},
\mathbf{L} = \begin{bmatrix}l_{11}&l_{12}&\ldots&l_{1n}\\l_{21}&\ddots& &l_{2n}\\\vdots& &\ddots& \vdots\\l_{n1}&\ldots&\ldots&l_{nn}\end{bmatrix}, \\
\mathbf{U} = \begin{bmatrix}u_{11}&u_{12}&\ldots&u_{1n}\\u_{21}&\ddots& &u_{2n}\\\vdots& &\ddots& \vdots\\u_{n1}&\ldots&\ldots&u_{nn}\end{bmatrix},
\mathbf{P} = \begin{bmatrix}p_{11}&p_{12}&\ldots&p_{1n}\\p_{21}&\ddots& &p_{2n}\\\vdots& &\ddots& \vdots\\p_{n1}&\ldots&\ldots&p_{nn}\end{bmatrix}
\end{equation}

O pseudocódigo para a fatoração PLU é o seguinte:

| Passos | |
| --:   | :-- |
|   1.  | Inicialize $\mathbf{L = P = I}$ de dimensão $n \times n$ e $\mathbf{U = A}$ |
|   2.  | Para $i = 1, \ldots, n$ realize passos 3-4, 8 |
|   3.  | $\phantom{--}$ Faça $k = i$, |
|   4.  | $\phantom{--}$ Enquanto $u_{ii}=0$, realize passos 5-7 |
|   5.  | $\phantom{----}$ Troque linha $U_i$ pela linha $U_{ k+1 }$ |
|   6.  | $\phantom{----}$ Troque linha $P_i$ pela linha $P_{ k+1 }$ |
|   7.  | $\phantom{----}$ Incremente $k$ em $1$. |
|   8.  | $\phantom{--}$ Para $j = i+1, \ldots, n$ realize passos 9-10 |
|   9.  | $\phantom{----}$ Faça $l_{ji} = u_{ ji }/ u_{ ii }$ |
|  10.  | $\phantom{----}$ Calcule $U_{ j }=U_{j} - l_{ji} U_{ i }$ (sendo $U_i, U_j$ as linhas $i$ e $j$ da matriz $\mathbf{U}$, respectivamente.)

### 6. Implementação em Python do Algoritmo PLU

In [44]:
import numpy as np

def plu(A):
    n = A.shape[0]
    L = np.eye(n, dtype=float)  # L começa como identidade
    U = np.zeros_like(A, dtype=float)  # U começa zerada
    P = np.eye(n, dtype=float)  # P começa como identidade

    # Cria uma cópia de A para não modificar original
    A = A.copy()

    for k in range(n):
        # Encontra pivô e realiza permutações
        max_row = np.argmax(np.abs(A[k:, k])) + k
        if np.isclose(A[max_row, k], 0):
            raise ValueError("Matriz singular, não pode ser decomposta.")

        if max_row != k:
            # Permuta A e P
            A[[k, max_row], :] = A[[max_row, k], :]
            P[[k, max_row], :] = P[[max_row, k], :]

            # Permuta apenas as colunas anteriores da matriz L
            if k > 0:
                L[[k, max_row], :k] = L[[max_row, k], :k]

        # Calcula linha k de U
        U[k, k:] = A[k, k:] - L[k, :k] @ U[:k, k:]

        # Calcula coluna k de L (abaixo do pivô)
        for i in range(k + 1, n):
            L[i, k] = (A[i, k] - L[i, :k] @ U[:k, k]) / U[k, k]

    return P, L, U

Expliquemos o comando:

In [45]:
A = np.array([[2, -1, -2],[-4, 6, 3],[-2, -7, 8]], dtype=float)
for k in range(3):
    max_row = np.argmax(np.abs(A[k:, k])) + k

max_row

np.int64(2)

Vamos analisar o código passo a passo para determinar o valor de `max_row` após a execução do laço `for k in range(3)` com a matriz do exemplo.

A matriz $ A $ é:

$$
A = \begin{bmatrix}
2 & -1 & -2 \\
-4 & 6 & 3 \\
-2 & -7 & 8
\end{bmatrix}
$$

O laço itera sobre $ k = 0, 1, 2 $, e em cada iteração, `max_row` é atualizado com o índice da linha que contém o maior valor absoluto na subcoluna $ A[k:, k] $ (a partir da linha $ k $ até o final da coluna $ k $), ajustado pelo deslocamento $ k $. Vamos calcular isso para cada valor de $ k $:

#### Iteração 1: $ k = 0 $
- Subcoluna: $ A[0:, 0] = [2, -4, -2] $ (coluna 0 inteira).
- Valores absolutos: $ |2| = 2 $, $ |-4| = 4 $, $ |-2| = 2 $.
- `np.argmax([2, 4, 2])` retorna $ 1 $ (índice do maior valor, 4).
- `max_row = 1 + 0 = 1`.

#### Iteração 2: $ k = 1 $
- Subcoluna: $ A[1:, 1] = [6, -7] $ (coluna 1 a partir da linha 1).
- Valores absolutos: $ |6| = 6 $, $ |-7| = 7 $.
- `np.argmax([6, 7])` retorna $ 1 $ (índice do maior valor, 7).
- `max_row = 1 + 1 = 2`.

####  Iteração 3 : $ k = 2 $
- Subcoluna: $ A[2:, 2] = [8] $ (coluna 2 a partir da linha 2, apenas um elemento).
- Valor absoluto: $ |8| = 8 $.
- `np.argmax([8])` retorna $ 0 $ (único elemento).
- `max_row = 0 + 2 = 2`.

Após o laço, o valor final de `max_row` é o calculado na última iteração, ou seja, $ k = 2 $.

O valor de `max_row` após a execução do código é:

\[
2
\]

Testemos tal implementação num exemplo simples:

In [46]:
import numpy as np

In [47]:
A = np.array([[2, -1, -2],[-4, 6, 3],[-2, -7, 8]], dtype=float)
[P,L,U] = plu(A)
P,L,U

(array([[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.]]),
 array([[ 1. ,  0. ,  0. ],
        [ 0.5,  1. ,  0. ],
        [-0.5, -0.2,  1. ]]),
 array([[ -4. ,   6. ,   3. ],
        [  0. , -10. ,   6.5],
        [  0. ,   0. ,   0.8]]))

Verifiquemos o resultado:

In [48]:
np.linalg.norm(P@A-L@U, ord=np.inf)

np.float64(5.551115123125783e-16)

Podemos agora construir a função que resolve um sistema linear:

In [52]:
def plu_solve(A, b):
    P, L, U = plu(A)
    y = GaussAv(L, np.dot(P, b))
    return GaussRetro(U, y)

Testemos em um exemplo:

In [53]:
A = np.array([[0.,4.,5.], [5.,9.,-3.],[3.,5.,1.]])
B = np.transpose(np.array([[2,3,1]]))

In [54]:
X1 = plu_solve(A,B)
X1

array([[-0.81818182],
       [ 0.72727273],
       [-0.18181818]])

Podemos verificar que esta é realmente a solução:

In [55]:
A@X1-B

array([[ 1.33226763e-15],
       [-4.44089210e-16],
       [-6.66133815e-16]])

### 7. PLU com Scipy

Há um pacote em Scipy capaz de resolver um sistema usando fatoração PLU:

In [94]:
from scipy.linalg import lu_factor, lu_solve
import numpy as np

A fatoração LU sem pivotação (ou seja, sem permutação de linhas) não é oferecida diretamente por funções padrão como `scipy.linalg.lu_factor`, pois elas fazem pivotação parcial automaticamente para garantir estabilidade numérica. Testemos este módulo num exemplo:

In [95]:
A = np.array([[0.,4.,5.,5.], [5.,9.,-3.,-1.],[2.,3.,5.,1.],[3,5.,6.,-3.]])
B = np.transpose(np.array([[2,3,1,3]]))

Inicialmente fatoramos a matriz

In [107]:
lu = lu_factor(A)
lu

(array([[ 0.9982214 ,  0.94106481,  0.3126623 , ...,  0.379088  ,
          0.82143508,  0.83598552],
        [ 0.13475251,  0.85361966,  0.08360687, ...,  0.31159159,
          0.81272568,  0.36827079],
        [ 0.05836445,  0.31200737,  0.89311874, ...,  0.05339477,
         -0.24450718,  0.5666222 ],
        ...,
        [ 0.00727189,  0.64266564,  0.30617851, ..., -1.88559639,
          0.6924629 ,  1.82833902],
        [ 0.39199257,  0.04722655,  0.16557447, ..., -0.16066823,
         -1.45419495,  0.79374839],
        [ 0.95769913, -0.73239905,  0.08554928, ..., -0.05530301,
          0.39095519,  1.01174516]]),
 array([40, 43, 10,  5, 36, 39, 83, 74, 14, 95, 87, 82, 60, 94, 95, 49, 78,
        60, 72, 47, 80, 35, 41, 25, 63, 38, 86, 27, 71, 43, 95, 95, 88, 91,
        58, 86, 65, 90, 93, 85, 88, 95, 93, 50, 63, 89, 65, 55, 50, 76, 81,
        62, 69, 61, 66, 84, 87, 64, 96, 64, 97, 82, 96, 84, 98, 76, 98, 90,
        79, 99, 77, 71, 82, 99, 92, 89, 90, 95, 89, 96, 93, 96, 94, 9

Façamos alguns comentários sobre tal saída. Analisemos inicialmente o vetor que descreve a pivotação:

In [97]:
lu[1]

array([1, 1, 3, 3], dtype=int32)

Isso significa que:

- Na etapa 0, a linha 0 foi trocada com a linha 1.

- Na etapa 1, a linha 1 permaneceu inalterada.

- Na etapa 2, a linha 2 foi trocada com a linha 3.

- Na etapa 3, a linha 3 permaneceu a mesma.

Por outro lado, consideremos a matriz de saída:

In [98]:
lu[0]

array([[ 5.        ,  9.        , -3.        , -1.        ],
       [ 0.        ,  4.        ,  5.        ,  5.        ],
       [ 0.6       , -0.1       ,  8.3       , -1.9       ],
       [ 0.4       , -0.15      ,  0.8373494 ,  3.74096386]])

 Esta saída é da forma [L\U], sendo que a parte abaixo da diagonal corresponde à matriz L e o resto é a matriz U. O próximo passo consiste em usar o comando lu_solve:

In [99]:
x = lu_solve(lu, B)
x

array([[-0.70853462],
       [ 0.7294686 ],
       [ 0.10305958],
       [-0.28663446]])

Calculemos o resíduo:

In [100]:
R = A@x-B
R

array([[0.0000000e+00],
       [4.4408921e-16],
       [0.0000000e+00],
       [8.8817842e-16]])

ou

In [101]:
import numpy.linalg as la

In [102]:
NR = la.norm(R, np.inf)
NR

np.float64(8.881784197001252e-16)

Vejamos o tempo aproximado de CPU para realizar este cálculo e façamos uma comparação com o cálculo sem pivotação. Usemos um sistema linear grande aleatório (para o o qual pivotação não é essencial):

In [103]:
# Tamanho do sistema
n = 100

# Gerar uma matriz A aleatória (com valores flutuantes)
A = np.random.rand(n, n)

# Gerar um vetor b aleatório
b = np.random.rand(n)
B = b.reshape(n,1)

Usemos inicialmente nosso algoritmo de fatoração LU, sem pivotação:

In [104]:
# Algoritmo sem pivotação:
%timeit lu_solve1(A,B)

17.7 ms ± 4.21 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [108]:
# 1. Fatoração LU com pivotação parcial
%%timeit
LU = lu_factor(A)
lu_solve(LU, b)

399 µs ± 89.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Como esperado, mesmo realizando pivotação, o código do sistema é muito mais rápido. A solução direta é um pouco mais rápida:

In [105]:
%timeit la.solve(A,B)

109 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### 8. Fatoração de Cholesky

O chamado método de Cholesky é um tipo de fatoração LU usando em alguns casos em que a matriz de coeficientes $A$ é simétrica. Uma matriz positivo definida se ela é simétrica e todos seus autovalores são positivos. Equivalentemente, uma matriz positivo definida é aquela na qual os pivots (os elementos na diagonal após o escalonamento) são positivos.

É possível mostrar que uma matriz $A$ positivo definida pode ser decomposta como

$$
A = G G^T,
$$

sendo $G$ a matriz triangular inferior com elementos da diagonal positivos.

Para construir o método de determinação de $G$ tomemos $n=4$:

\begin{equation}
\mathbf{A} =
\begin{pmatrix}
a_{11}&a_{12}&a_{13}&a_{14}\\
a_{21}&a_{22}&a_{23}&a_{24}\\
a_{31}&a_{32}&a_{33}&a_{34}\\
a_{41}&a_{42}&a_{43}&a_{44}
\end{pmatrix}
  =
  \begin{pmatrix}
g_{11}&0&0&0\\
g_{21}&g_{22}&0&0\\
g_{31}&g_{32}&g_{34}&0\\
g_{41}&g_{42}&g_{43}&g_{44}
\end{pmatrix}
  \begin{pmatrix}
g_{11}&g_{21}&g_{31}&g_{14}\\
0&g_{22}&g_{32}&g_{42}\\
0&0&g_{33}&g_{43}\\
0&0&0&g_{44}
\end{pmatrix}
\end{equation}

Portanto, para os elementos da diagonal temos

\begin{eqnarray}
a_{11} = &g_{11}^2\\
a_{22}=& \,g_{21}^2+g_{22}^2\\
a_{33}=& \,g_{31}^2+g_{32}^2+g_{33}^2\\
a_{44}=& \,g_{41}^2+g_{42}^2+g_{43}^2+g_{44}^2 .
\end{eqnarray}

De modo geral, para uma dimensão $n$ qualquer,

$$
a_{nn} = g_{n1}^2+g_{n2}^2+\cdots + g_{nn}^2
$$

Portanto,

$$
g_{11}=\left(a_{11}\right)^{1/2},
$$


$$
g_{ii}=\left(a_{ii}-\sum_{k=1}^{i-1}g_{kk}^2\right)^{1/2}\,,\qquad i=2, \ldots, n\,.
$$

Para os elementos fora da diagonal (basta levar em conta as componentes abaixo da diagonal) temos:

(i) Primeira coluna:
\begin{eqnarray}
a_{21} = &g_{21}g_{11}\\
a_{31} = &g_{31}g_{11}\\
a_{41} = &g_{41}g_{11}\\
\end{eqnarray}

de modo geral,

$$
g_{i1}=\frac{a_{i1}}{g_{11}}\,,\qquad i=2, \ldots , n\,.
$$

(ii) Segunda coluna:

\begin{eqnarray}
a_{32} = &g_{31}g_{21}+g_{32}g_{22}\\
a_{42} = &g_{41}g_{21}+g_{42}g_{22}.
\end{eqnarray}

De modo geral,

$$
a_{i2}=g_{i1}g_{21}+g_{i2}g_{22},\qquad i=3, \ldots, n\,.
$$

ou

$$
g_{i2}=\frac{a_{i2}-g_{i1}g_{21}}{g_{22}},\qquad i=3, \ldots, n\,.
$$

(iii) Terceira coluna:

$$
a_{43}= g_{41}g_{31}+g_{42}g_{32}+g_{43}g_{33}\,,
$$

de modo que

$$
g_{43}=\frac{a_{43}-g_{41}g_{31}-g_{42}g_{32}}{g_{33}}\,.
$$

Em geral,

$$
g_{ij} = \frac{a_{ij}-\sum_{k=1}^{j-1}g_{ik}g_{jk}}{g_{jj}}\,,\qquad j=2,\ldots, i-1\,.
$$

Implementemos este método:

In [ ]:
def cholesky(a):
  n = a.shape[0]
  a = a.copy()
  g = np.zeros((n,n), dtype = np.double)
  for k in range(n):
    g[k,k] = np.sqrt(a[k,k] - np.sum(g[k,:k]**2))
    for i in range(k+1,n):
       g[i,k] = (a[i,k]-np.sum(g[i,:k]*g[k,:k]))/g[k,k]
  return g

Testemos esta implementação:

In [ ]:
import numpy as np

In [ ]:
A = np.array([[2, -1, 1],[-1, 2, -1.], [1, -1, 2.]])
A

array([[ 2., -1.,  1.],
       [-1.,  2., -1.],
       [ 1., -1.,  2.]])

In [ ]:
G = cholesky(A)
G

array([[ 1.41421356,  0.        ,  0.        ],
       [-0.70710678,  1.22474487,  0.        ],
       [ 0.70710678, -0.40824829,  1.15470054]])

De fato,

In [ ]:
G@G.T-A

array([[ 4.4408921e-16,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00, -4.4408921e-16,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  4.4408921e-16]])

Usando o comando do numpy:

In [ ]:
import numpy.linalg as la

In [ ]:
G2 = la.cholesky(A)
G2

array([[ 1.41421356,  0.        ,  0.        ],
       [-0.70710678,  1.22474487,  0.        ],
       [ 0.70710678, -0.40824829,  1.15470054]])

Antes de tentar resolver um sistema usando o método de Cholesky devemos testar se esta matriz é simétrica e positivo definida:

In [ ]:
import numpy as np

def simetrica_positiva_definida(A, tol=1e-10):
    # Verifica simetria
    if not np.allclose(A, A.T, atol=tol):
        return False, "Matriz não é simétrica."

    # Calcula autovalores
    autovalores = np.linalg.eigvalsh(A)  # mais eficiente para simétricas

    # Verifica se todos os autovalores são positivos
    if np.all(autovalores > tol):
        return True, "Matriz é simétrica e definida positiva."
    else:
        return False, "Matriz é simétrica, mas não definida positiva."

Vejamos exemplos:

In [ ]:
A = np.array([[2, -1, 1],[-1, 2, -1.], [1, -1, 2.]])
simetrica_positiva_definida(A)

(True, 'Matriz é simétrica e definida positiva.')

In [ ]:
A = np.array([[2, -1, 6],[-1, 2, -1.], [1, -1, 2.]])
simetrica_positiva_definida(A)

(False, 'Matriz não é simétrica.')

In [ ]:
A = np.array([[0.1, -1, 1],[-1, 2, -1.], [1, -1, 2.]])
simetrica_positiva_definida(A)

(False, 'Matriz é simétrica, mas não definida positiva.')

### 9. Fatoração QR

A chamada decomposição (ou fatoração) QR de uma matriz, é uma decomposição da forma:

$$
A = QR,
$$

sendo $Q$ uma matriz ortogonal ($QQ^T = I$) e $R$ uma matriz triangular superior.

Esta decomposição será usada  para resolver o problemas envolvendo o método de mínimos quadrados a no problema de determinação de autovalores e autovetores.

Se $A$ é não singular é possível mostrar que esta fatoração é única. Um dos métodos práticos para realizar a fatoração $QR$ é com o uso do método de ortogonalização de Gram-Schmidt, que descreveremos a seguir. Vamos escrever a matriz $A$, de ordem $n \times n$ na seguinte forma

$$
A = [a_1\,|\, a_2\,|\, a_3, \cdots |a_n],
$$

sendo $a_i = [a_{i1} a_{i2} a_{i3} \cdots a_{in}]^T$ o vetor correspondente à i-ésima coluna de $A$. A ideia do método consiste em usar o fato de que, como $A$ é não singular por hipótese, os vetores $\{a_i, \, i=1\, \ldots, n\}$ definem uma base. Ao definir outra base $\{u_i/\|u_i\|, \, i=1\, \ldots, n\}$ pelo processo de Gram-Schmidt, agora ortonormal, definimos a matriz ortogonal

$$
Q = [e_1\,|\, e_2\,|\, e_3| \cdots |e_n].
$$

O processo de Gram-Schmidt é iniciado a seguir:

$$
u_1=a_1,\,\qquad e_1= \frac{u_1}{\|u_1\|}.
$$

$$
u_2 = a_2 - (a_2 \cdot e_1)e_1\,\qquad e_2= \frac{u_2}{\|u_2\|}.
$$

Notemos que

$$
u_1 \cdot u_2 =u_1\cdot a_2 - (a_2 \cdot e_1)(\underbrace{u_1\cdot e_1}_{\|u_1\|})=0\,.
$$

Prosseguindo,

$$
u_3 = a_3 - (a_3 \cdot e_1)e_1 - (a_3 \cdot e_2)e_2\,\qquad e_3= \frac{u_3}{\|u_3\|}.
$$

O último vetor é da forma

$$
u_n = a_n - (a_n \cdot e_1)e_1 - (a_n \cdot e_2)e_2 - \cdots - (a_n \cdot e_{n-1})e_{n-1}\,\qquad e_n= \frac{u_n}{\|u_n\|}.
$$

Podemos agora reescrever $A$ na forma:

$$
A = [a_1\,|\, a_2\,|\, a_3, \cdots |a_n]=
[e_1\,|\, e_2\,|\, e_3| \cdots |e_n]\begin{pmatrix}a_1\cdot e_1& a_2\cdot e_1&
a_3\cdot e_1& \cdots &a_n \cdot e_1\\
0 & a_2\cdot e_2&
a_3\cdot e_2& \cdots &a_n \cdot e_2\\
0& 0&a_3\cdot e_3& \cdots &a_n \cdot e_3\\
\vdots& \vdots&\vdots& \cdots &\vdots\\
0& 0&0& \cdots &a_n \cdot e_n
\end{pmatrix}.
$$

Vejamos um exemplo:

In [ ]:
import numpy as np
import numpy.linalg as la

In [ ]:
A = np.array([[2.,4.,3.], [-4.,3.,7.], [-1.,2., 9.]])
A

array([[ 2.,  4.,  3.],
       [-4.,  3.,  7.],
       [-1.,  2.,  9.]])

Aqui

In [ ]:
a1 = A[:,0]
a2 = A[:,1]
a3 = A[:,2]
print(a1)
print(a2)
print(a3)

[ 2. -4. -1.]
[4. 3. 2.]
[3. 7. 9.]


In [ ]:
u1 = a1
e1 = u1/la.norm(u1)
e1

array([ 0.43643578, -0.87287156, -0.21821789])

In [ ]:
u2 = a2-np.dot(a2,e1)*e1
e2 = u2/la.norm(u2)
e2

array([0.87515358, 0.35553114, 0.32818259])

In [ ]:
u3 = a3-np.dot(a3,e1)*e1-np.dot(a3,e2)*e2
e3 = u3/la.norm(u3)
e3

array([-0.208878  , -0.33420479,  0.91906318])

In [ ]:
Q1= np.vstack(([e1],[e2],[e3]))
Q = Q1.T
Q

array([[ 0.43643578,  0.87515358, -0.208878  ],
       [-0.87287156,  0.35553114, -0.33420479],
       [-0.21821789,  0.32818259,  0.91906318]])

Podemos verificar que $Q$ é realmente ortogonal:

In [ ]:
Q.T@Q

array([[1.00000000e+00, 4.06255488e-17, 4.51017873e-17],
       [4.06255488e-17, 1.00000000e+00, 2.78432194e-16],
       [4.51017873e-17, 2.78432194e-16, 1.00000000e+00]])

In [ ]:
R = np.array([[np.dot(a1,e1),np.dot(a2,e1), np.dot(a3,e1)],
              [0,np.dot(a2,e2), np.dot(a3,e2)],
               [0,0, np.dot(a3,e3)]])
R

array([[ 4.58257569, -1.30930734, -6.7647546 ],
       [ 0.        ,  5.22357294,  8.06782208],
       [ 0.        ,  0.        ,  5.30550111]])

Podemos verificar que esta decomposição é correta:

In [ ]:
Q@R-A

array([[ 4.44089210e-16, -4.44089210e-16, -4.44089210e-16],
       [-8.88178420e-16,  0.00000000e+00, -8.88178420e-16],
       [-2.22044605e-16,  0.00000000e+00,  1.77635684e-15]])

Podemos obter o mesmo resultado usando comandos da biblioteca Scipy:

In [ ]:
import pprint
import numpy.linalg as la

In [ ]:
A = np.array([[2.,4.,3.], [-4.,3.,7.], [-1.,2., 9.]])

In [ ]:
Q, R = la.qr(A)

In [ ]:
print("A:")
pprint.pprint(A)

print("Q:")
pprint.pprint(Q)

print("R:")
pprint.pprint(R)

A:
array([[ 2.,  4.,  3.],
       [-4.,  3.,  7.],
       [-1.,  2.,  9.]])
Q:
array([[-0.43643578, -0.87515358, -0.208878  ],
       [ 0.87287156, -0.35553114, -0.33420479],
       [ 0.21821789, -0.32818259,  0.91906318]])
R:
array([[-4.58257569,  1.30930734,  6.7647546 ],
       [ 0.        , -5.22357294, -8.06782208],
       [ 0.        ,  0.        ,  5.30550111]])


In [ ]:
result = wikipedia.summary("QR decomposition")
print(result)

In linear algebra, a QR decomposition, also known as a QR factorization or QU factorization, is a decomposition of a matrix A into a product A = QR of an orthogonal matrix Q and an upper triangular matrix R. QR decomposition is often used to solve the linear least squares problem and is the basis for a particular eigenvalue algorithm, the QR algorithm.


### 10. Exercícios

**1.** Resolva passo a passo, usando o método LU de Doolittle, 0 sistema aleatório gerado usando os seguinte comandos do numpy:

In [ ]:
np.random.seed(22222) #use os primeiros 5 números do seu cpf
N = 3
A = np.random.rand(N,N)
B = np.random.rand(N,1)

**2.** Construa uma procedimento para inverter uma matriz qualquer, usando fatoração LU de Doolittle. Use ou modifique a função lu_solve definida acima. Teste o procedimento.

**3.** Resolva passo a passo  o sistema $AX = B$, usando fatoração LU, com

In [ ]:
A = np.array([[0.,3.,6.,1.], [1.,5.,-5.,6],[4.,7.,-1.,5.],[-2., 4., 9., -3]])
B = np.transpose(np.array([[-2.,4.,2.,5.]]))

**4.** Resolva o sistema do problema 3 usando a função python plu construída acima.

**5.** Verifique o quanto o tempo de CPU muda usando *pivotação* na função de fatoração LU do scipy. Use um sistema aleatório de 500 linhas para fazer o teste.

**6.** Gere uma matriz positivo definida de ordem $n \times n$ da seguinte forma: gere uma matriz aleatória $M$ e depois defina uma matriz positivo definida $A=M M^T$. Estude o desempenho dos algoritmos LU e Cholesky para para resolver sistemas lineares da forma $AX=B$. Examine o tempo de execução para diversos valores grandes de $n$.

**7.** Defina uma matriz positivo definida aleatória de ordem 4 e defina um sistema linear qualquer da forma $AX=B$. Resolva o sistema usando o método de Cholesky, passo a passo (o Python pode ser usado passo a passo).

**8.** Faça a decomposição $QR$ da matriz

$$
\begin{pmatrix}
3& -5&2&3\\
-3&8&2&1\\
6&7&21&-3
\end{pmatrix}
$$

**9.** Faça a decomposição QR de uma matriz aleatória $100 \times 100$ usando comandos do sistema.